In [24]:
# Loading packages
import numpy as np
import itertools
from pyscf import gto, ao2mo, scf
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev



In [109]:
# mol_h2o = gto.M(atom = 'H 0 0 0; H 0 0 .714', basis = '6-31G')
mol_h2o = gto.M(atom = 'H 0 0 0; H 0 0 .714', basis = 'sto-3G')

In [111]:
# Obtaining the optimal distance
conversion = 1.8897259885789 # 1 Angstrom is equal to 'conversion' atomic units
R = 0.7414

# Defining the molecule
mol = gto.Mole()
mol.build(
    atom = '''
        H 0 0 0; 
        H 0 0 0.7413773258490092
    ''',  # in Angstrom
    basis = 'sto-3G',
)

# restricted Hartree Fock
rhf_sols = scf.RHF(mol)
E_HF = rhf_sols.kernel()

# One and two electron integrals in terms of the atomic orbitals
hcore_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
coulomb_ao = mol.intor('int2e')

# Rearranging these in terms of the molecular orbitals
hcore_mo = np.einsum('pi,pq,qj->ij', rhf_sols.mo_coeff, hcore_ao, rhf_sols.mo_coeff)
coulomb_mo = ao2mo.incore.full(coulomb_ao, rhf_sols.mo_coeff)


# Wait, we got the indexing wrong, of course there is no index 3, we are dealing with 2x2 systems without the spi
# So indeed, we should change the second code a bit for it to work
# Make the indices run from 0 to 3 and the matrix indices from 0 to two
# Make it so we can not have 
# remember the factor 1/2 for the possibility of multiple configurations 
# it does seem plausible to get the right Hamiltonian
# Also make some boundary conditions such that we exlude the cases in which we have a double numbered operator on the same basis


converged SCF energy = -1.11668562734567


In [117]:
# E_HF-1/(conversion*R)
# Now we want to construct the Hamiltonian from the integral terms.
# What would be the best way to approach this problem
# The matrices do not contain the spin yet, maybe we need to add this by extending the matrices?

# Maybe we could do it with a unch of if statements? Seems like the best choice overall
# Pair this with a mod statement!

# Defining number of particles and the possible iterations
N = len(hcore_ao[0,:])  # Number of molecular orbitals (excluding spin)
two_terms = list(itertools.product([ i for i in range(2*N) ], repeat=2))

# Initializing the Hamiltonian
hamiltonian_fermionic_op = 0

for indices in two_terms:

    # Obtaining the spin indices
    spin_indices = np.array(indices) % N

    # Adding terms to the matrix
    if np.all(spin_indices == 0) or np.all(spin_indices == 1):

        # Obtaining matrix indices to index 
        matrix_indices = np.array(indices) // N
        
        # Adding terms to the Hamiltonian
        hamiltonian_fermionic_op += FermionOperator(
            str(indices[0])+'^ '+str(indices[1]), 
            hcore_mo[matrix_indices[0], matrix_indices[1]]
        )


# Doing the same for the coulomb terms (should be exactly the same except for the addition of the string and the division by two)
# four_terms = list(itertools.product([ i for i in range(2*N) ], repeat=4))
four_terms = list(itertools.product([ 0, 2], repeat=4))
hamiltonian_fermionic_op = 0

for indices in four_terms:

    if np.any([np.all(np.array(indices) == j) for j in range(2*N) ]) == False:

        if indices[0] != indices[1] and indices[2] != indices[3]:

            # Obtaining the spin indices
            spin_indices = np.array(indices) % N

            # Adding terms to the matrix
            if spin_indices[0] == spin_indices[3] and spin_indices[1] == spin_indices[2]:

                # Obtaining matrix indices to index 
                matrix_indices = np.array(indices) // N
                
                # Adding terms to the Hamiltonian
                hamiltonian_fermionic_op += FermionOperator(
                    str(indices[0])+'^ '+str(indices[1])+'^ '+str(indices[2])+ ' '+ str(indices[3])+ ' ', 
                    1/2*coulomb_mo[matrix_indices[0], matrix_indices[1], matrix_indices[2], matrix_indices[3]]
                )

In [118]:
hamiltonian_fermionic_op

0.09064376939955535 [0^ 2^ 0 2] +
0.09064376939955542 [0^ 2^ 2 0] +
0.09064376939955535 [2^ 0^ 0 2] +
0.09064376939955543 [2^ 0^ 2 0]

In [9]:
from openfermion.chem import MolecularData
from openfermion import FermionOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermionpyscf import run_pyscf
from hamiltonian.matrix_hamiltonian import get_matrix_form
import numpy as np


# Set parameters to make a simple molecule.
diatomic_bond_length = .7414
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = '3-21G'
# basis = 'sto-3G'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)

# Initializing the openfermionpyscf software
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule,
                        run_scf=run_scf,
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)

In [10]:
# Obtaining the fermionic Hamiltonian
molecular_hamiltonian = molecule.get_molecular_hamiltonian()
fermionic_hamiltonian = get_fermion_operator(molecular_hamiltonian)

# Removing the nuclear repuslion 
fermionic_hamiltonian += -FermionOperator('', fermionic_hamiltonian.terms[()])

# Obtaiing the Jordan-Wigner Hamiltonian
qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)
# bravyi_kitaev(fermionic_hamiltonian)

In [11]:
# We can actually optimize it using only one, two, three and four neirest neighbouring parties
H = get_matrix_form(8, qubit_hamiltonian)
eig_vals, eig_vecs = np.linalg.eigh(H)
eig_vals[0]

-1.8615843126111304

In [13]:
# Let's see if we can njit the classical optimization function, that part seems to be the troublesome function
# It heavily bottlenecks the capabilities of the system, maybe we can speed it up.
# I do not think the itertools is a valid njit function, some maybe we do need to change that one

(1.8661950186883578+0j) [] +
(-0.022282700268445258+0j) [X0 X1 Y2 Y3] +
(-0.020025172578589576+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.020025172578589576+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.028545460511678968+0j) [X0 X1 Y4 Y5] +
(-0.03277269075771477+0j) [X0 X1 Y6 Y7] +
(0.022282700268445258+0j) [X0 Y1 Y2 X3] +
(0.020025172578589576+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.020025172578589576+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.028545460511678968+0j) [X0 Y1 Y4 X5] +
(0.03277269075771477+0j) [X0 Y1 Y6 X7] +
(-0.004162288539263977+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.004162288539263977+0j) [X0 Z1 X2 Y3 Z4 Y5] +
(0.016594860837795128+0j) [X0 Z1 X2 X4 Z5 X6] +
(0.003946015238577306+0j) [X0 Z1 X2 Y4 Z5 Y6] +
(0.023349354365647692+0j) [X0 Z1 X2 X5 Z6 X7] +
(0.023349354365647692+0j) [X0 Z1 X2 Y5 Z6 Y7] +
(0.012648845599217817+0j) [X0 Z1 Y2 Y4 Z5 X6] +
(-0.019403339127070388+0j) [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
(-0.006754493527852571+0j) [X0 Z1 Z2 X3 X5 X6] +
(0.019403339127070388+0j) [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
(-0.00675449352